<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>

# 1. Explore the Data with *pandas*
<hr>

Electronic Health Records (EHRs) contain a wealth of patient medical information that can:
* Save valuable time when an emergency arises
* Eliminate unnecesary treatment and tests
* Prevent potentially life-threatening mistakes
* Improve the overall quality of care a patient receives when seeking medical assistance

Children's Hospital Los Angeles (CHLA) wanted to know if the records could be mined to yield early warning signs of patients that may require extra care or an indication of the severity of a patient's illness.  In this lab we have access to the work and results of CHLA's applied use of deep neural networks on EHRs belonging to roughly 5,000 pediatric ICU patients.

We will start by exploring the data using the python library [*pandas*](http://pandas.pydata.org) to manage the dataset provided in [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) format.

1.1 [**Set Up the Environment**](#01_setup)<br>
1.2 [**Load the Data with *pandas***](#01_pandas)<br>
1.3 [**Visualize the Data**](#01_explore)<br>
&nbsp; &nbsp; &nbsp;1.3.1 [Example: View Data from a Single Encounter](#01_single)<br>
&nbsp; &nbsp; &nbsp;1.3.2 [Exercise: Observation Count Histogram](#01_ex_nobs)<br>
&nbsp; &nbsp; &nbsp;1.3.3 [Exercise: Length of PICU Stay Histogram](#01_ex_time)<br>
1.4 [**Save a *pandas* DataFrame**](#01_save)<br>
[**Solutions**](#01_solutions)<br>

<a name="01_setup"></a>
## 1.1 Set Up the Environment

First, import the libraries we need into our Python workspace.  We need *os* for access to the file system,  *NumPy* for fast array math, *pandas* for data management, and  *MatPlotLib* for visualization.<br>
Execute the cell below to import these libraries and define the location of the data in the file system.

In [ ]:
import os
import numpy as np
import pandas as pd              
import matplotlib.pyplot as plt 
import random

# Configure the notebook to display plots
%matplotlib inline

# Define the path to the data folder
data_dir = '/dli/task/data/hx_series'

Next, specify the file paths in the data folder which contains training and validation datasets stored in HDF5 format: inputs (X) and their associated labels (y).
[HDF5](http://www.hdfgroup.org/) stands for "hierarchical data format version number 5".  The HDF format is designed specifically to store and organize large amounts of scientific data and was originally designed by the [National Center for Supercomputing Applications](https://en.wikipedia.org/wiki/National_Center_for_Supercomputing_Applications).  Common file extensions include `.hdf`, `.hdf5`, or simply `.h5`.  The HDF format has become very popular and is well maintained.  As a result, HDF5 is a flexible and robust format having API support in most languages and library compatibilty with Windows, OS X and Linux. It is important to note that HDF is a binary format and hence lacks the human readable transparency of text based CSV files.  However, HDF file format has faster performance and more efficient storage size.  It also scales well from small proof-of-concept ideas to [very large operational projects](https://www.hdfgroup.org/eos_vignette/).

In [ ]:
# training data inputs: x and targets: y
x_train_path = os.path.join(data_dir, 'X_train.hdf')
y_train_path = os.path.join(data_dir, 'y_train.hdf')

# validation data inputs: x and targest: y
x_valid_path = os.path.join(data_dir, 'X_test.hdf')
y_valid_path = os.path.join(data_dir, 'y_test.hdf')

<a name="01_pandas"></a>
## 1.2 Load the Data with <i>pandas</i>
Finally, we load the data using the *pandas* API for reading in HDF files.  Python with pandas is used in a wide variety of academic and commercial domains, including finance, neuroscience, economics, statistics, advertising, web analytics, and more. The pandas library is an open source, BSD-licensed project providing easy-to-use data structures and analysis tools for the Python programming language. The pandas library features a fast and efficient DataFrame object for data manipulation with integrated indexing as well as tools for reading and writing data between in-memory data structures and different formats such as CSV and text files, Microsoft Excel, SQL databases, and the fast HDF5 format. Check out the [pandas documentation](http://pandas.pydata.org) for more info.

In [ ]:
# It should take less than one minute to load the files
X_train = pd.read_hdf(x_train_path)
y_train = pd.read_hdf(y_train_path)
X_valid = pd.read_hdf(x_valid_path)
y_valid = pd.read_hdf(y_valid_path)
print('data load complete')

Note:  An alternative to *pandas* is [*cuDF*](https://github.com/rapidsai/cudf/tree/master), a Python GPU DataFrame library (built on the Apache Arrow columnar memory format) for loading, joining, aggregating, filtering, and otherwise manipulating data, all in a *pandas*-like API familiar to data scientists.  For larger data sets especially, this is an alternative you may wish to try for faster processing.

<a name="01_explore"></a>
## 1.3 Visualize the Data

This electronic health records (EHR) database contains medical treatments and histories of patients collected over time. The EHRs used here consists of 10 years worth of patient encounter data in the Pediatric Intensive Care Unit (PICU) at Children's Hospital Los Angeles, curated by the virtual PICU ([vPICU](http://vpicu.net)) team. This dataset contains 1,113,529 observations among 5,000 unique patient encounters. 

<img style="float: right;" src="images/ehr.svg" width="450" height="300">
This data is an irregular time series of observations consisting of varied numbers of measurements taken over the course of a patient's stay in the PICU. Time between observations can vary from minutes to hours. A simplified diagram of the data can be seen on the right. <br>
Features (measurement types) include:

* **Statics** *(e.g. gender, age, weight)*
* **Vitals** *(e.g. heart rate, respiratory rate)*
* **Labs** *(e.g. glucose, creatinine)*
* **Interventions** *(e.g. intubation, O2)*
* **Drugs** *(e.g. dopamine, epinephrine)*

For a complete list of features, see [ehr_features.csv](csv/ehr_features.csv)

One thing to note is that in addition to the non-uniform sampling, not all measurement types were taken for all patients.

If we just have a look at the training data, it's clear that we have a collection of patient encounters with a set of variables observed at different times during each encounter.  But again, not all variables are measured at each observation time (row entry). 

The label (y) data for each patient encounter is the ultimate result of alive (1) or not alive (0). 

Let's take a look at the data.

In [ ]:
X_train

Notice here that there are 265 measurement variables (columns) in total.  We could also ask directly using `len(X_train.columns)`.<br>
The data imported by is a multi-index DataFrame where index "level 0" is the unique patient encounter identifier (`encounterID`) and index "level 1" is the time of each observation in units of hours since the first observation (`absoluteTime`).

Have a quick look at the label data for each patient encounter.  We see it follows the same indexing, but only contains a single binary value for each observation.

In [ ]:
y_train

<a name='01_single'></a>
### 1.3.1 Example: View Data from a Single Encounter

We can look more closely at a single patient visit by specifying the `encounterID` "level 0" index.  To extract just these values, try the following:

In [ ]:
X_train.index.levels[0]

The _pandas_ DataFrame is easily converted to a regular Python list using `list()`, as shown in the code following code block.  To demonstrate how to manipulate DataFrame objects, we will select a random patient encounter and extract specific variables.  Run the next cell a few times to get a feel for the variety of data associated with various random patients:

In [ ]:
# Select a random patient encounterID from a list of all the encounterID values
eIdx = random.choice(list(X_train.index.levels[0]))

# Specify a few variables to look at
variables = [
    'Age','Heart rate (bpm)','PulseOximetry','Weight',
    'SystolicBP','DiastolicBP','Respiratory rate (bpm)',
    'MotorResponse','Capillary refill rate (sec)'
]

# Note that the full list of variables can be constructed using
# list(X_train.columns.values)

# Have a look at the variables for the selected patient
print('encounterID = {}'.format(eIdx))
print('number of observations = {}'.format(X_train.loc[eIdx].index.shape[0]))
print('max absoluteTime value = {} hours'.format(X_train.loc[eIdx].index[-1]))
X_train.loc[eIdx, variables]

The number of rows tells us how many observations are included in the patient encounter, and the final `absoluteTime` indicates how many hours long the entire encounter lasted.

To get a look at a variable for a particular patient encounter simply extract that variable from an encounter and plot it using the _pandas_ plot function.

In [ ]:
X_train.loc[eIdx, "Heart rate (bpm)"].plot()
plt.ylabel("Heart rate (bpm)")
plt.xlabel("Hours since first encounter")
plt.show()

<a name="01_ex_nobs"></a>
### 1.3.2 Exercise: Observation Count Histogram

We might want to know more about the variability of the time series data. For example, how many observations are there in an encounter typically?  A histogram will provide a good visualization of how the variability in observations per patient is distributed.  To create a histogram we need to:

1. Create a list of `encounterID` values
2. Create a list of observation counts for each of these encounters
3. Create a histogram using MatplotLib  

In [ ]:
# TODO Step 1 - Create a list of `encounterID` values
# Hint: In the single encounter example, we created this list and selected a random choice from it
eIdx_list = [] #FIXME
print('eIdx_list created of length {} (Sanity check: this length should be 5000)'.format(len(eIdx_list)))

In [ ]:
# Step 2 - Create a list of observation counts for each of these encounters
# The number of observations in each encounter can be extracted as the shape of each individual encounter
nobs_list = [X_train.loc[ix].index.shape[0] for ix in eIdx_list]
print('list of observation counts created of length {} (Sanity check: this length should also be 5000)'.format(len(nobs_list)))

In [ ]:
# Step 3 - Create a histogram using MatplotLib
plt.hist(nobs_list,range=(0,1000))
plt.title("Observations per Patient Encounter")
plt.show()

If you've done created the initial eIdx list correctly, you should see a histogram (if not, check the [solution](#01_sol_nobs) below).  Now you can also check the mean and median counts using `np.mean()` and `np.median()` functions over the list. 

In [ ]:
print('The mean = {}'.format(np.mean(nobs_list)))
print('The median = {}'.format(np.median(nobs_list)))

<a name='01_ex_time'></a>
### 1.3.3 Exercise: Length of PICU Stay Histogram
We can do a similar analysis to determine the observation timespan over all patient encounters.  Recall that the observation index `absoluteTime` is the cumulative time since admission to the PICU.  Therefore, the last `absoluteTime` value is the one we want to capture for each encounter.  Earlier in the section, we displayed this value for an individual encounter using `X_train.loc[eIdx].index[-1]`.  

Give it a try.  You can always check the [solution](#01_solution_1.3.2) if you wish.

1. Create a list of `encounterID` values
2. Create a list of `absoluteTime` final for each encounter
3. Create a histogram using MatplotLib  

In [ ]:
# TODO Step 1 - Create a list of `encounterID` values
# Hint: Earlier in the section, we created this list and selected a random choice from it
eIdx_list = [] #FIXME
print('eIdx_list created of length {} (Sanity check: this length should be 5000)'.format(len(eIdx_list)))

In [ ]:
# TODO Step 2 - Create a list of `absoluteTime` final for each encounter
# Hint: Use a Python list comprehension over the eIdx_list
timespan_list = [] #FIXME
print('timespan_list created of length {} (Sanity check: this length should be 5000)'.format(len(timespan_list)))

In [ ]:
# Step 3 - Create a histogram using MatplotLib
plt.hist(timespan_list,range=(0,500))
plt.title("Hours per Patient Encounter")
plt.show()

If you've done created the initial eIdx list correctly, you should see a histogram (if not, check the [solution](#01_sol_time) below).  Now you can also check the mean and median counts using `np.mean()` and `np.median()` functions over the list. 

In [ ]:
print('The mean = {}'.format(np.mean(timespan_list)))
print('The median = {}'.format(np.median(timespan_list)))

<a name="01_save"></a>
## 1.4 Save a *pandas* DataFrames

Before moving to the next notebook, save the *pandas* DataFrames to reload into the next notebook.

In [ ]:
# Save the DataFrame's for use in other notebooks
X_train.to_pickle('X_train.pkl')
y_train.to_pickle('y_train.pkl')
X_valid.to_pickle('X_valid.pkl')
y_valid.to_pickle('y_valid.pkl')

<font color='green' size=10>Great job!</font><br>
Now that we've loaded and visualized the data, we'll prepare it to train our model.  Move on to the [next notebook](02_PrepareData.ipynb).

<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>

<hr>

<a name='01_solutions'></a>
# Solutions
<a name='01_sol_nobs'></a>
### 1.3.2 Solution: Observation Count Histogram 
[Jump back to Exercise 1.3.2](#01_ex_nobs)

In [ ]:
# TODO Step 1 - Create a list of `encounterID` values
# Hint: In the single encounter example, we created this list and selected a random choice from it
eIdx_list = X_train.index.levels[0] #FIXME
print('eIdx_list created of length {} (Sanity check: this length should be 5000)'.format(len(eIdx_list)))

In [ ]:
# Step 2 - Create a list of observation counts for each of these encounters
# The number of observations in each encounter can be extracted as the shape of each individual encounter
nobs_list = [X_train.loc[ix].index.shape[0] for ix in eIdx_list]
print('list of observation counts created of length {} (Sanity check: this length should also be 5000)'.format(len(nobs_list)))

In [ ]:
# Step 3 - Create a histogram using MatplotLib
plt.hist(nobs_list,range=(0,1000))
plt.title("Observations per Patient Encounter")
plt.show()

<a name='01_sol_time'></a>
### 1.3.3 Solution: Length of PICU Stay Histogram
[Jump back to Exercise 1.3.3](#01_ex_time)

In [ ]:
# TODO Step 1 - Create a list of `encounterID` values
# Hint: Earlier in the section, we created this list and selected a random choice from it
eIdx_list = X_train.index.levels[0] #FIXME
print('eIdx_list created of length {} (Sanity check: this length should be 5000)'.format(len(eIdx_list)))

In [ ]:
# TODO Step 2 - Create a list of `absoluteTime` final for each encounter
# Hint: Use a Python list comprehension over the eIdx_list
timespan_list = [X_train.loc[ix].index[-1] for ix in eIdx_list] #FIXME
print('timespan_list created of length {} (Sanity check: this length should be 5000)'.format(len(timespan_list)))

In [ ]:
# Step 3 - Create a histogram using MatplotLib
plt.hist(timespan_list,range=(0,500))
plt.title("Hours per Patient Encounter")
plt.show()